In [93]:
# read brand names into a list
brand_file = open('brands.txt', 'r')
brands = brand_file.readlines()

for i in range(0, len(brands)):
    brands[i] = brands[i][:-1]

In [105]:
import os,sys
import pandas as pd
import re
import glob
import random

inputDir = 'selected_texts/training_texts/'
inputFiles = glob.glob(inputDir + '*.txt')

output = []

for inputFile in inputFiles:
    print(inputFile)
    with open(inputFile,'r') as f:
        text = f.read()
        words = text.split(' ')

        start_index = [0]
        end_index = []

        for m in re.finditer('<p>(.+?)</>',text):
            end_index.append(m.start())
            start_index.append(m.end())

        end_index.append(len(text))

        text_fragments = []
        
        for i in range(0, len(start_index)):
            text_fragments.append(text[start_index[i] : end_index[i]])

        file_name = inputFile[30:]
        candidates = []
        n = 8
            
        for i, fragment in enumerate(text_fragments):
            fragment = fragment.split(' ')

            if '' in fragment:
                fragment.remove('')

            for start in range(0, len(fragment)):
                for length in range(2, 8):
                    if (start + length) <= len(fragment):
                        candidate_string = fragment[start : start + length]
                        candidate_string = ' '.join(s for s in candidate_string)

                        before = text.find(candidate_string)
                        after = before + len(candidate_string)

                        text_before = text[:before].split(' ')
                        text_after = text[after:].split(' ')

                        for t in range(0, len(text_before)):
                            text_before[t] = text_before[t].replace('<p>', '')
                            text_before[t] = text_before[t].replace('</>', '')

                        for t in range(0, len(text_after)):
                            text_after[t] = text_after[t].replace('<p>', '')
                            text_after[t] = text_after[t].replace('</>', '')

                        candidate_up = ' '.join(s for s in text_before[-n:])
                        candidate_down = ' '.join(s for s in text_after[:n])
                        candidate_follow = candidate_string[-1:]

                        candidates.append([candidate_string, candidate_up, candidate_down, candidate_follow])
                        
        negative_examples = []

        for candidate in candidates:
            candidate_string = candidate[0]
            words = candidate_string.split(' ')
            
            satisfied = 0

            # criterion 1: Are more than half of the words capitalized?
            found = 0
            for word in words:
                if word.isalpha() and word[0].isupper():
                    found += 1
            
            if found / len(words) >= 0.5:
                satisfied += 1

            # criterion 2: Does any word contain digit(s)?
            found = False
            for word in words:
                if any(char.isdigit() for char in word):
                    found = True
                    break

            if found:
                satisfied += 1

            # criterion 3: Does any word contain '(', ')', '/', '-', '"' or '\''?
            found = False
            for word in words:
                if ('(' in word) or (')' in word) or ('/' in word) or ('-' in word) or ('"' in word) or ('\'' in word):
                    found = True
                    break

            if found:
                satisfied += 1
            
            # criterion 4: Does the string starts with a brand name?
            found = False
            for brand in brands:
                if words[0].lower() == brand.lower():
                    found = True
                    break
            
            if found:
                satisfied += 1
            
            if satisfied == 4:
                negative_examples.append(candidate)
            
    output.extend(negative_examples)

print(len(output))   

selected_texts/training_texts/Acer AM3470-UC30P.txt
selected_texts/training_texts/Acer Aspire A5600U-UB13 .txt
selected_texts/training_texts/Acer Aspire AS1830T-3935 (AT&T).txt
selected_texts/training_texts/Acer Aspire AS5742-7645 .txt
selected_texts/training_texts/Acer Aspire AS5742Z-4097.txt
selected_texts/training_texts/Acer Aspire One.txt
selected_texts/training_texts/Acer Aspire Predator AG7750-U2222.txt
selected_texts/training_texts/Acer Aspire Predator G7710-U7790A.txt
selected_texts/training_texts/Acer Aspire S3-391-6046.txt
selected_texts/training_texts/Acer Aspire S7-392-6411.txt
selected_texts/training_texts/Acer Aspire S7-393-7451.txt
selected_texts/training_texts/Acer Aspire Switch 10 E (SW3-013-11N8).txt
selected_texts/training_texts/Acer Aspire Switch 10.txt
selected_texts/training_texts/Acer Aspire V3-772G-9460.txt
selected_texts/training_texts/Acer Aspire V5-571P-6627.txt
selected_texts/training_texts/Acer AspireRevo R3610-U9022.txt
selected_texts/training_texts/Acer C

In [115]:
import csv

csv_f = open('neg_feat_vec.csv', 'w')

# header of the table
fields = ['product_name','num_of_words', 'total_str_len', 'avg_word_len', 'fraction_capitalized', 'num_of_non-Eng_words', 'num_of_digits', 'word(s)_with_uppercase_letters', 'starts_with_brand', 'paranthesis', ')_in_last_word', 
          '\w-\w_end', '\w-\w_second', '\d_end', 'contains_year', 'contains_inch_info', 'contains_core_info', 'starts_with_i', 'concatenated_words', 'keywords_downstream', 
          'is_was_downstream', '(_downstream', '$_downstream', 'the_upstream', ',_or_._after_string', 'label']
writer = csv.writer(csv_f, delimiter = ',')
writer.writerow(fields)

fvs = []

for example in output:
    product_name = example[0]
    inst_up = example[1].split(' ')
    inst_down = example[2].split(' ')
    inst_follow = example[3]
    
    # add product name at the front
    vector = []
    vector.append(product_name)

    # feature 1(a): number of words
    # feature 1(b): total number of characters in the string (excluding blank space)
    # feature 1(c): average word length
    words = product_name.split()
    vector.append(len(words))
    
    total_len = 0
    
    for word in words:
        total_len += len(word)
    
    vector.append(total_len)
    vector.append(total_len / len(words))

    # feature 2: What is the fraction of capitalized words?
    # feature 3: How many words are not English words?
    upper = 0
    total = 0
    
    for word in words:
        if word.isalpha():
            total += 1
            if word[0].isupper():
                upper += 1
    
    vector.append(upper / total)
    vector.append(len(words) - total)

    # feature 4: How many digits are in the string?
    found = 0
    for word in words:
        for char in word:
            if char.isdigit():
                found += 1

    vector.append(found)

    # feature 5: Is there a word whose characters are all capitalized (excluding the leading word)?
    found = False
    for word in words[1:]:      # skip 1st word
        if word.isalpha() and word.isupper():
            found = True
            break

    if found:
        vector.append('T')
    else:
        vector.append('F')

    # feature 6: Does the string start with a brand name, and the brand name only appears once?
    found = False
    
    if words[0].lower() in brands:
        found = True
    
    for word in words[1:]:
        if word.lower() in brands:
            found = False
    
    if found:
        vector.append('T')
    else:
        vector.append('F')

    # feature 7(a): Does the string contain '(' and ')'?
    # feature 7(b): Does ')' appear in the last word as the last character?
    found1 = False
    found2 = False
    pos = -1
    
    for i, word in enumerate(words):
        if '(' in word:
            found1 = True
        if ')' in word:
            found2 = True

    if found1 and found2:
        vector.append('T')
    else:
        vector.append('F')
    
    last_word = words[len(words) - 1]
    
    if last_word[-1:] == ')':
        vector.append('T')
    else:
        vector.append('F')

    # feature 8(a): Does the last word contain a string of alphanumeric characters delimited by '-' ?
    # feature 8(b): Does the second word contain a string of alphanumeric characters delimited by '-'?
    # feature 8(c): Does the last word contain a number?
    found1 = False
    found2 = False
    found3 = False
    
    if len(re.findall('\w+-\w+', words[len(words) - 1])) > 0:
        found1 = True
    
    if len(re.findall('\w+-\w+', words[1])) > 0:
        found2 = True
    
    if len(re.findall('\d+', words[len(words) - 1])) > 0:
        found3 = True

    if found1:
        vector.append('T')
    else:
        vector.append('F')
    
    if found2:
        vector.append('T')
    else:
        vector.append('F')
        
    if found3:
        vector.append('T')
    else:
        vector.append('F')

    # feature 9: Does any word contain a year?
    found = False
    for word in words:
        if len(re.findall('20[0-1][0-7]', word)) > 0:
            found = True
            break

    if found:
        vector.append('T')
    else:
        vector.append('F')

    # feature 10: Does the string contain information on "inches"?
    found = False
    for word in words:
        word = word.lower()
        if ('inch' in word.lower()) or ('inches' in word.lower()) or ('"' in word.lower()):
            found = True
            break

    if found:
        vector.append('T')
    else:
        vector.append('F')

    # feature 11: Does the string contain information on "core"?
    found = False
    for word in words:
        if 'core' in word.lower():
            found = True
            break

    if found:
        vector.append('T')
    else:
        vector.append('F')

    # feature 12: Is there a word that starts with 'i'?
    found = False
    for word in words:
        if word.startswith('i'):
            found = True
            break

    if found:
        vector.append('T')
    else:
        vector.append('F')

    # feature 13: Are there two capitalized words concatenated into one word?
    found = False
    for word in words:
        if (len(re.findall('[A-Z][a-z]+', word))) > 1:
            found = True
            break

    if found:
        vector.append('T')
    else:
        vector.append('F')

    # feature 14: Is there a keyword immediately downstream?
    found = False
    for word in inst_down:
        word = word.lower()
        if word.find('laptop') != -1 or word.find('desktop') != -1 or word.find('tablet') != -1 or word.find('notebook') != -1:
            found = True
            break

    if found:
        vector.append('T')
    else:
        vector.append('F')

    # feature 15: Does 'is' or 'was' occur immediately downstream?
    found = False
    for word in inst_down[:2]:
        if word.lower() in ['is', 'was']:
            found = True
            break

    if found:
        vector.append('T')
    else:
        vector.append('F')

    # feature 16: Does '(' and/or ')' occur immediately downstream?
    found = False
    for word in inst_down[:2]:
        if '(' in word.lower():
            found = True
            break

    if found:
        vector.append('T')
    else:
        vector.append('F')

    # feature 17: Does '$' occur immediately downstream?
    found = False
    for word in inst_down[:2]:
        if '$' in word.lower():
            found = True
            break

    if found:
        vector.append('T')
    else:
        vector.append('F')

    # feature 18: Does 'the' occur immediately upstream?
    found = False
    for word in inst_up[-2:]:
        if word.lower() == 'the':
            found = True
            break

    if found:
        vector.append('T')
    else:
        vector.append('F')

    # feature 19: Does ',' or '.' occur immediately after the string?
    found = False
    if (inst_follow == ',') or (inst_follow == '.'):
        found = True

    if found:
        vector.append('T')
    else:
        vector.append('F')
    
    vector.append('NEG')

    writer.writerow(vector)   
    fvs.append(vector)

csv_f.close()